<a href="https://www.kaggle.com/code/yanelly/querywhiz-ai-powered-data-questions-insights?scriptVersionId=235358536" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🤖 GenAI for Data Analysis: Ask Questions, Get SQL, Understand Results

Welcome! This project demonstrates how to use **Google Gemini AI** to enhance the data analysis process with:
- AI-generated **questions and SQL queries**
- Natural language **summaries of query results**
- Support for **custom CSV uploads** or a built-in **Kaggle dataset**

🎯 Goal: Show how GenAI can act like a smart analyst assistant for any dataset.


# 1. 🔧 Setup: Libraries, API Key, and Imports

In [2]:
# Gemini + API Setup
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
import warnings

In [3]:
# For API retry handling
from google.api_core import retry

In [4]:
# Displaying Data
import pandas as pd

In [5]:
# SQLite DB
import sqlite3

In [6]:
# Librairies for the inputed csv:
from IPython.display import display
import ipywidgets as widgets
import io

In [7]:
# API Error 
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [8]:
# getting API key
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

# 2. 📂 Load a Dataset (Kaggle OR Upload your own CSV)

## Loading kaggle data

In [9]:
# Data Set: 
data = pd.read_csv('/kaggle/input/amazon-sales-2025/amazon_sales_data 2025.csv')
data

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending
...,...,...,...,...,...,...,...,...,...,...,...
245,ORD0246,17-03-25,T-Shirt,Clothing,20,2,40,Daniel Harris,Miami,Debit Card,Cancelled
246,ORD0247,30-03-25,Jeans,Clothing,40,1,40,Sophia Miller,Dallas,Debit Card,Cancelled
247,ORD0248,05-03-25,T-Shirt,Clothing,20,2,40,Chris White,Denver,Debit Card,Cancelled
248,ORD0249,08-03-25,Smartwatch,Electronics,150,3,450,Emily Johnson,New York,Debit Card,Cancelled


## Loading input data

> 👉 **Tip**: You can upload your own `.csv` file using the widget below. If you skip this step, the notebook will use a default Amazon Sales dataset.


In [10]:
# Upload CSV widget
upload = widgets.FileUpload(accept='.csv', multiple=False)
display(upload)

# Handle upload
def handle_upload():
    if upload.value:
        for filename in upload.value:
            content = upload.value[filename]['content']
            df = pd.read_csv(io.BytesIO(content))
            df.columns = [col.strip().replace(" ", "_") for col in df.columns]
            return df
    return None  # No file uploaded

# Try user upload
df = handle_upload()

# Fallback if nothing is uploaded
if df is None:
    print("⚠️ No user file uploaded — using default Kaggle dataset instead.")
    df = pd.read_csv('/kaggle/input/amazon-sales-2025/amazon_sales_data 2025.csv')
    df.columns = [col.strip().replace(" ", "_") for col in df.columns]


FileUpload(value=(), accept='.csv', description='Upload')

⚠️ No user file uploaded — using default Kaggle dataset instead.


# 3. 🧠 Ask AI: What Should We Explore?

## prompt for the kaggle data set

In [11]:
# Prompt:
sample = data.head(5).to_markdown()  # Only show a small sample in the prompt
prompt = f"""
Here is a a few rows of our dataset:

{sample}

Based on this dataset, what are some useful questions we should ask during further data analysis?
"""

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents=prompt
)
print(response.text)

Okay, here are some useful questions we should ask based on the provided dataset, categorized for clarity:

**I. Sales Performance & Trends:**

*   **Overall Sales Performance:**
    *   What is the total revenue generated?
    *   What is the average order value?
    *   How many orders were placed in total?
*   **Time-Based Analysis:**
    *   What are the sales trends over time (daily, weekly, monthly)?  Are there any seasonal patterns?
    *   Which dates/months had the highest and lowest sales?  Why?
*   **Product Analysis:**
    *   Which product is the best seller? Which is the worst?
    *   What is the average quantity sold per product?
    *   What is the sales distribution across different product categories?
    *   Are there any products that are frequently purchased together? (Good for suggesting product bundles)
*   **Customer Analysis


## Prompt for the input data

In [12]:
# Prompt:
sample_1 = df.head(5).to_markdown()

prompt_1 = f"""
Here is a few rows of our dataset:

{sample_1}

Based on this dataset, what are some useful questions we should ask during further data analysis?
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents=prompt_1
)

print(response.text)


Okay, based on the dataset provided, here are some useful questions to ask during further data analysis, categorized for clarity:

**1. Sales Performance & Trends:**

*   **Overall Sales:**
    *   What is the total sales revenue generated over the entire period?
    *   What is the average order value?
    *   What is the distribution of order values? (Are most orders small, or are there a few large orders driving revenue?)
*   **Temporal Trends:**
    *   How do sales fluctuate over time (daily, weekly, monthly)? Are there any seasonal trends?
    *   Is there any growth or decline in sales over time?
    *   Are there specific dates or periods with unusually high or low sales? (e.g., promotions, holidays)
*   **Product Performance:**
    *   Which products are the best-selling?
    *   Which categories are the most popular?
    *   


# 4. 🧾 Generate & Run SQL Queries from Natural Language

In [13]:
# description function for both kaggle and user inputed csv:
def describe_table(conn, table_name: str):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    return [(col[1], col[2]) for col in cursor.fetchall()]

In [14]:
# Query function for both kaggle and user inputed csv: 
def execute_query(conn, sql: str) -> list[list[str]]:
    print(f' - DB CALL: execute_query({sql})')
    cursor = conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

## running SQL Queries for the kaggle data set

In [15]:
# Using sqlite3 to create the database
kaggle_conn = sqlite3.connect("sample.db")
data.columns = [col.strip().replace(" ", "_") for col in data.columns]
data.to_sql("data", kaggle_conn, if_exists="replace", index=False)

250

In [18]:
describe_table(kaggle_conn, "data")

[('Order_ID', 'TEXT'),
 ('Date', 'TEXT'),
 ('Product', 'TEXT'),
 ('Category', 'TEXT'),
 ('Price', 'INTEGER'),
 ('Quantity', 'INTEGER'),
 ('Total_Sales', 'INTEGER'),
 ('Customer_Name', 'TEXT'),
 ('Customer_Location', 'TEXT'),
 ('Payment_Method', 'TEXT'),
 ('Status', 'TEXT')]

In [19]:
execute_query(kaggle_conn,"select * from data where Category == 'Footwear'")

 - DB CALL: execute_query(select * from data where Category == 'Footwear')


[('ORD0001',
  '14-03-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Emma Clark',
  'New York',
  'Debit Card',
  'Cancelled'),
 ('ORD0003',
  '15-02-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'John Doe',
  'Denver',
  'Amazon Pay',
  'Cancelled'),
 ('ORD0004',
  '19-02-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Olivia Wilson',
  'Dallas',
  'Credit Card',
  'Pending'),
 ('ORD0019',
  '22-03-25',
  'Running Shoes',
  'Footwear',
  60,
  3,
  180,
  'Olivia Wilson',
  'Houston',
  'Credit Card',
  'Completed'),
 ('ORD0046',
  '06-03-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'David Lee',
  'Houston',
  'Debit Card',
  'Cancelled'),
 ('ORD0053',
  '24-03-25',
  'Running Shoes',
  'Footwear',
  60,
  4,
  240,
  'Emily Johnson',
  'Los Angeles',
  'PayPal',
  'Completed'),
 ('ORD0079',
  '09-03-25',
  'Running Shoes',
  'Footwear',
  60,
  2,
  120,
  'Emily Johnson',
  'Denver',
  'Gift Card',
  'Cancelled'),
 ('ORD0080',
  '23-02

## Running SQL Queries from the input data

In [20]:
# 🛢️ Save uploaded data to SQLite
user_conn = sqlite3.connect("sample_1.db")
df.columns = [col.strip().replace(" ", "_") for col in df.columns]
df.to_sql("df", user_conn, if_exists="replace", index=False)

250

In [21]:
describe_table(user_conn,"df")

[('Order_ID', 'TEXT'),
 ('Date', 'TEXT'),
 ('Product', 'TEXT'),
 ('Category', 'TEXT'),
 ('Price', 'INTEGER'),
 ('Quantity', 'INTEGER'),
 ('Total_Sales', 'INTEGER'),
 ('Customer_Name', 'TEXT'),
 ('Customer_Location', 'TEXT'),
 ('Payment_Method', 'TEXT'),
 ('Status', 'TEXT')]

In [22]:
"""Uncomment the line below to execute a query on the user-uploaded dataset."""
# execute_query()

'Uncomment the line below to execute a query on the user-uploaded dataset.'

In [23]:
print("📂 Default dataset loaded into SQLite as 'data'")
print("📂 User-uploaded dataset loaded into SQLite as 'df'")

📂 Default dataset loaded into SQLite as 'data'
📂 User-uploaded dataset loaded into SQLite as 'df'


# Agents

In [24]:
# Agents function for both Kaggle and user-uploaded CSV:
def agent_loop(df, conn, table_name="data"):
    print("🔍 Ask a question about the dataset (or type 'exit'):")

    # Dynamically build schema from df
    schema = f"Table: {table_name}\nColumns:\n"
    for col in df.columns:
        schema += f"- {col}\n"

    # Start interaction loop
    while True:
        user_input = input("\n🧍 You: ")
        if user_input.lower() == "exit":
            break

        # Prompt for SQL
        prompt = f"""
        You are a helpful assistant that answers data questions by generating SQL queries.
        Here is the table schema:
        {schema}
        Question: {user_input}
        Only respond with a valid SQL query.
        """
        
        # Gemini API call using the Kaggle client
        response = client.models.generate_content(  # Make sure `client` is initialized properly
            model='gemini-2.0-flash',
            config=short_config,
            contents=prompt
        )
        sql = response.text.strip().replace("```sql", "").replace("```", "").strip()
        print(f"\n🧾 Cleaned SQL:\n{sql}")

        try:
            result = execute_query(conn, sql)  # Assuming `conn` is a valid connection object
            print("\n📊 Query Results:")
            for row in result:
                print(row)

            # Explanation
            summary_prompt = f"""
            Here is the result of the SQL query:
            {result}
            Explain this result in plain English for a data analyst.
            """
            summary = client.models.generate_content(  # Again, using the Kaggle client here
                model='gemini-2.0-flash',
                config=short_config,
                contents=summary_prompt
            )
            print(f"\n🗣️ Summary:\n{summary.text}")
        except Exception as e:
            print(f"❌ Error: {e}")


## 5. 🤖 Agent Mode: Chat with Your Data W kaggle data and input data

In [25]:
agent_loop(data, kaggle_conn) 

🔍 Ask a question about the dataset (or type 'exit'):



🧍 You:   What is the average order value?



🧾 Cleaned SQL:
SELECT AVG(Total_Sales) AS Average_Order_Value FROM data;
 - DB CALL: execute_query(SELECT AVG(Total_Sales) AS Average_Order_Value FROM data;)

📊 Query Results:
(975.38,)

🗣️ Summary:
Okay, here's the explanation for a data analyst:

"The SQL query returned a single result: a single row containing one value, which is 975.38.  We can interpret this as a single aggregate value calculated based on the data in the table(s) queried.  For example, it could be the sum, average, maximum, or minimum of a particular column, or the result of a more complex calculation. Without knowing the specifics of the query, the best interpretation is that **the query resulted in a single number, 975.38, which likely represents some aggregate statistic or a calculated value.**"

In short: It's a single numerical result, likely an aggregated value from your data.  You'd need to look at the SQL query itself to understand what this number actually *means* in the context of your data.




🧍 You:  exit


In [26]:
"""Uncomment the line below to execute the agent_loop on the user-uploaded dataset."""
# agent_loop(df, user_conn, table_name="df")

'Uncomment the line below to execute the agent_loop on the user-uploaded dataset.'